您已經建立了一個模型。但這模型品質多好？

在本課程中，會學習使用模型驗證(model validation)來度量模型的質量。測量模型品質是不斷改進模型的關鍵。

# 什麼是模型驗證 What is Model Validation

您幾乎會評估曾經構建的每個模型。在大多數（儘管不是全部）應用中，模型質量的相關度量是**預測準確性**。換句話說，模型的預測將接近實際發生的情況。

許多人在測量預測準確性時犯了一個巨大的錯誤。他們使用其訓練數據進行預測，並將這些預測與訓練數據中的目標值進行比較。我們將很快看到這種方法的問題以及如何解決它，但是讓我們考慮一下我們將如何進行此操作。

我們首先需要以一種易於理解的方式來總結模型質量。如果您比較10,000套房屋的預測房屋價格和實際房屋價值，則可能會發現混合在一起好壞預測結果。瀏覽10,000個預測值和實際值的列表將毫無意義。我們需要將其匯總為一個指標。

概述模型品質有很多指標，但我們將從一個稱為 **平均絕對誤差** （Mean Absolute Error，**MAE**）的指標開始。讓我們從最後一個詞“誤差(error)”開始分解這個指標。

每個房屋的預測誤差為：

```python
error=actual−predicted
```

因此，如果一房屋的價格為 150,000，而您預測這將花費 100,000，則誤差為 50,000。

使用 MAE 度量，我們可以獲取每個誤差的絕對值。 這會將每個錯誤轉換為正數。 然後，我們取這些絕對誤差的平均值。 這是我們對模型質量的衡量。 簡單來說，可以說是:

> 就平均而言，我們的預測相差約 X。

要計算MAE，我們首先需要一個模型。 它內置在下面的隱藏cell中，您可以通過單擊 `code` 按鈕進行查看。

In [1]:
# Data Loading Code Hidden Here
import pandas as pd

# Load data
melbourne_file_path = './data_set/melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path) 
# Filter rows with missing price values
filtered_melbourne_data = melbourne_data.dropna(axis=0)
# Choose target and features
y = filtered_melbourne_data.Price
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea', 
                        'YearBuilt', 'Lattitude', 'Longtitude']
X = filtered_melbourne_data[melbourne_features]

from sklearn.tree import DecisionTreeRegressor
# Define model
melbourne_model = DecisionTreeRegressor()
# Fit model
melbourne_model.fit(X, y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

一旦有了模型，就可以計算平均絕對誤差：

In [2]:
from sklearn.metrics import mean_absolute_error

predicted_home_prices = melbourne_model.predict(X)
mean_absolute_error(y, predicted_home_prices)

434.71594577146544

## “樣本中(In-Sample)”分數的問題

我們剛剛計算出的度量可以稱為“樣本中”分數。我們使用單個房屋“樣本”來構建模型和評估模型。這就是為什麼這很糟糕。

想像一下，在大型房地產市場中，門的顏色與房價無關。

但是，在用於構建模型的數據樣本中，所有帶有綠色門的房屋都非常昂貴。該模型的工作是找到可以預測房價的模式，因此它將看到此模式，並且始終可以預測帶有綠色門的房屋具有高價格。

由於此模式是從訓練數據中得出的，因此模型在訓練數據中將顯得準確。

但是，如果在模型看到新數據時這種模式不成立，那麼在測試資料集的表現在使用該模型將表現的非常不准確。

由於模型的實際價值來自對新數據的預測，因此我們可以用來評估未用於構建模型的數據。最簡單的方法是從模型構建過程中排除一些數據，然後使用這些數據對以前從未見過的數據測試模型的準確性。該數據稱為 **驗證數據(validation data)**。

## Coding It

scikit-learn函式庫具有一個函數 - `train_test_split` ，可將數據分為兩部分。 我們將使用其中一些數據作為適合模型的訓練數據，並使用其他數據作為驗證數據來計算  `mean_absolute_error` 。

以下是程式碼：

In [3]:
from sklearn.model_selection import train_test_split

# split data into training and validation data, for both features and target
# The split is based on a random number generator. Supplying a numeric value to
# the random_state argument guarantees we get the same split every time we
# run this script.
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)
# Define model
melbourne_model = DecisionTreeRegressor()
# Fit model
melbourne_model.fit(train_X, train_y)

# get predicted prices on validation data
val_predictions = melbourne_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

261452.58295674628


## 哇！

樣本中數據的平均絕對誤差約為500美元。 訓練集資料外平均絕對誤差超過25萬美元。

這就是幾乎完全正確的模型與無法用於大多數實際目的的模型之間的差異。作為參考，驗證數據中的平均房屋價值為110萬美元。 因此，新數據中的誤差約為平均房屋價值的四分之一。

有許多方法可以改善此模型，例如嘗試尋找更好的功能或不同的模型類型。